# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter von Domherren einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: 
`domherr.tbl_domherren` und `domherr.tbl_domherren_aemter`

[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Archidiakonat](#Archidiakonat)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt) (hinfällig?)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

*2022-11-03* optimiere das Datenupdate; Test mit `wiag_in´  

In [9]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [26]:
item_type_id = 5

5

Evtl. neue Quellen eintragen!

In [10]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-11-22"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\domherren-2022-11-22"

In [11]:
out_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data_sql"

Die Ämter in Tabelle `role` werden im Notebook "Strukturdaten" eingelesen.

## Amtsperiode

In [12]:
using CSV, DataFrames

In [13]:
# Quelle CSV
filename = joinpath(data_path, "tbl_Domherren_Aemter.csv")
df_pr = CSV.read(filename, DataFrame);

In [11]:
# Quelle Datenbank
sql = "SELECT * FROM domherr.tbl_domherren_aemter"
df_pr = Wds.sql_df(sql);

In [14]:
Wds.clean_up!(df_pr);

Ignoriere Einträge wo sowohl Amtsart als auch ID_Amt_norm fehlen.

In [15]:
size(df_pr)

(40244, 20)

In [16]:
one_valid(a, b) = !ismissing(a) | !ismissing(b)

one_valid (generic function with 1 method)

In [17]:
subset!(df_pr, [:Amtsart, :ID_Amt_norm] => ByRow(one_valid));

In [18]:
df_pr[101:105, [:ID_Amt, :Amtsart, :ID_Amt_norm, :Amtsbeginn, :Amtsende]]

Row,ID_Amt,Amtsart,ID_Amt_norm,Amtsbeginn,Amtsende
,Int64,String?,Int64?,String?,String?
1,93022,missing,19,1456/1459,1463
2,93023,missing,20,1458,missing
3,93024,missing,20,1458,missing
4,93025,missing,19,1495/1515,1558
5,93026,missing,64,1539,1558


### Domherren zuordnen

In [27]:
transform!(df_pr, :ID_Domherr => ByRow(string) => :id_in_source);

In [28]:
table_name = "item";
sql = "SELECT id AS person_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)"
df_idx = Wds.sql_df(sql);

Lösche Einträge ohne Wert für `ID_Domherr`

In [29]:
dropmissing!(df_pr, :ID_Domherr);

In [30]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

In [31]:
df_idx_mg = filter(:person_id => ismissing, df_prp);

In [32]:
size(df_idx_mg)

(0, 22)

In [33]:
df_idx_mg[:, [:ID_Domherr, :ID_Amt, :Amtsart]]

0×3 DataFrame

### Rollen/Ämter zuordnen

In [34]:
sql = "SELECT name as role_name, role.id as role_id, item.id_in_source as role_id_in_source
FROM role JOIN item ON item.id = role.id"
df_role = Wds.sql_df(sql);

Werden alle Ämter gefunden?

In [35]:
role_dh = unique(string.(df_prp.ID_Amt_norm));

In [36]:
match_mg = setdiff(role_dh, df_role.role_id_in_source)

1-element Vector{Union{Missing, String}}:
 "missing"

In [37]:
to_string_maybe(a) = ismissing(a) ? a : string(a)

to_string_maybe (generic function with 1 method)

In [38]:
transform!(df_prp, :ID_Amt_norm => ByRow(to_string_maybe) => :match_role_id);

In [39]:
df_prp_r = leftjoin(df_prp, df_role, on = :match_role_id => :role_id_in_source, matchmissing = :notequal);

In [40]:
count(ismissing, df_prp_r.role_id), count(ismissing, df_prp_r.Amtsart)

(1158, 8376)

In [41]:
df_prp_r_mg_rid = filter(:role_id => ismissing, df_prp_r);

In [42]:
unique(df_prp_r_mg_rid.Amtsart)

85-element Vector{Union{Missing, String}}:
 "Dombaumeister"
 "Wirtschaftsverwalter"
 "Inhaber der officium minor"
 "officiales"
 "Domprior/seit seit 1504 Dekane"
 "Domprior/seit 1504 Dekan"
 "Mitglied"
 "Ecclesiasticus"
 "Domkustoden/Domthesaurare"
 "Präsenzmeister"
 "Präsenzkeller"
 "Providierter Domherr/Domscholast"
 "Kapellar"
 ⋮
 "vorwarer"
 "Vorsteher, Amtmann, confessarius, Gemeindepfarrer"
 "vormund"
 "bischöflicher Vogt"
 "Amtsverwalter"
 "Domina bzw. Äbtissin"
 "Drost u. Amtsschreiber"
 "Amtspächter bzw. -verwalter"
 "Lehnpropst"
 "Vorsteher (Ordenskaplan)"
 "Weltgeistlicher"
 "Rectores scholarum"

*2022-10-19* Was machen wir mit diesen Einträgen?

*2022-03-16* TODO übernimm die Rollenbezeichnung `person_role.role_name` aus der Tabelle `role`.

### Bistümer zuordnen

In [43]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

In [44]:
df_prp_r = leftjoin(df_prp_r, df_dioc, on = :Bistum => :diocese_name, matchmissing = :notequal);

In [45]:
idx_diocese = .!ismissing.(df_prp_r[!, :diocese_id]);

In [46]:
sum(idx_diocese)

1090

### Instutionen zuordnen
Klöster, Domstifte

In [47]:
table_name = "institution";
sql = "SELECT id AS institution_id, name, id_gsn FROM $(table_name)"
df_inst = Wds.sql_df(sql);

In [48]:
df_prp_r = leftjoin(df_prp_r, df_inst, on = :ID_Kloster => :id_gsn, matchmissing = :notequal);

Version mit eigener lookup-Funktion

In [49]:
lookup_inst = Dict(df_inst.id_gsn .=> df_inst.institution_id);

In [50]:
length(lookup_inst)

5966

In [51]:
get_inst_id(id_gsn) = get(lookup_inst, id_gsn, missing)

get_inst_id (generic function with 1 method)

In [52]:
get_inst_id(318)

79619

In [53]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_id) => :institution_id);

In [54]:
lookup_inst_name = Dict(df_inst.id_gsn .=> df_inst.name);

In [55]:
get_inst_name(id_gsn) = get(lookup_inst_name, id_gsn, missing)

get_inst_name (generic function with 1 method)

In [56]:
get_inst_name(318)

"Benediktinerinnenkloster Neuenwalde"

In [57]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_name) => :institution_name);

In [58]:
df_mg_inst_id = subset(df_prp_r, :institution_id => ByRow(ismissing));

In [59]:
size(df_mg_inst_id, 1), size(df_prp_r, 1)

(979, 39454)

### Numerische Daten eintragen

In [60]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
df_prp_r[!, :num_date_begin] .= num_date_begin.(df_prp_r[!, :Amtsbeginn]);

┌ Warning: Could only find year in 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1125
┌ Warning: Could only find year in 
│   s = t 1567
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = Bewerbung 1451
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = [1068]
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = erneut 1239
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte

In [61]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper)
df_prp_r[!, :num_date_end] .= num_date_end.(df_prp_r[!, :Amtsende]);

┌ Warning: Could only find year in 
│   s = und 1118
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = t1529
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = t1547
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = und 1118
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could only find year in 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1120
┌ Warning: Could not parse 
│   s = 66
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src

In [62]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [63]:
size(df_info, 1)

31078

In [58]:
df_info[60:67, [:ID_Domherr, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

Row,ID_Domherr,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int64,String,String?,Int64?,String?,Int64?
1,14112,Domkustos (Domthesaurar),1744,1744,1757,1757
2,14113,Domkustos (Domthesaurar),1778,1778,1792,1792
3,14115,Domkustos (Domthesaurar),1792,1792,1811,1811
4,14117,Domkantor,vor 1328,1278,missing,missing
5,14118,Domscholaster,vor 1328,1278,missing,missing
6,14122,Domkantor,1376,1376,1376,1376
7,14124,Domkantor,1405,1405,1419,1419
8,14128,Domkantor,1452,1452,1455,1455


In [64]:
size(df_prp)

(39454, 23)

*2022-05-20* Datenfehler in der Quelle korrigiert

In [56]:
function fix_date(s)
    if ismissing(s)
        return s
    end
    return replace(s, "1 Hälfte" => "1. Hälfte", "14 Jahr" => "14. Jahr", "Beginn16" => "Beginn 16")
end

fix_date (generic function with 1 method)

In [57]:
transform!(df_prp_r, :Amtsbeginn => ByRow(fix_date) => :Amtsbeginn);

In [58]:
transform!(df_prp_r, :Amtsende => ByRow(fix_date) => :Amtsende);

In [65]:
transform!(df_prp_r, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1553) 1560
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1574) 1593
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = t 1567
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (Bewerbung 1451)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = [1068]
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539


┌ Warning: could not parse 
│   s = (1553) 1560
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539


*2022-05-12* In `tbl_Domherren_Aemter` enthalten 200 Einträge ein `†`. Das wird vor dem Parsen entfernt.

In [66]:
transform!(df_prp_r, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: could not parse 
│   s = und 1118
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = t1529
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = t1547
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = und 1118
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = [1241]
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not 

In [67]:
maximum(df_prp_r.date_sort_key)

9000900

In [68]:
replace_if_missing(a, b) = a == 9000900 ? b : a

replace_if_missing (generic function with 1 method)

In [69]:
transform!(df_prp_r, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

In [70]:
size(df_prp_r, 1), count(isequal(9000900), df_prp_r.date_sort_key)

(39454, 1358)

In [71]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [72]:
df_info[100:105, [:Amtsart, :ID_Amt_norm, :role_id, :Amtsbeginn, :num_date_begin, :num_date_end, :date_sort_key]]

Row,Amtsart,ID_Amt_norm,role_id,Amtsbeginn,num_date_begin,num_date_end,date_sort_key
,String,Int64?,Int32?,String?,Int64?,Int64?,Int64
1,Archidiakon,5,489576,1727,1727,1742,1727150
2,Archidiakon,5,489576,vor 1707,1657,missing,1707100
3,Archidiakon,5,489576,vor 1755,1705,missing,1755100
4,Archidiakon,5,489576,1712,1712,1753,1712150
5,Archidiakon,5,489576,1784,1784,1791,1784150
6,Archidiakon,5,489576,1791,1791,1792,1791150


Einträge für diesen Item_Typ löschen

In [73]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_id))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 39454, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer - auch für die identifizierten - den Namen des Bistums in die Tabelle.  
Schreibe für alle Institutionen - auch für die identifizieren - die Bezeichnung in die Tabelle.  
Schreibe für alle Ämter - auch für die identifizieren - die Bezeichnung in die Tabelle.

Erzeuge die IDs selbst, um sie für `person_role_property` zur Verfügung zu haben.

In [74]:
table_name = "person_role"
sql = "SELECT MAX(id) + 1 AS next_id FROM $(table_name)"
df_next_id = Wds.sql_df(sql)

Row,next_id
,Int64?
1,1003158


In [75]:
next_id = df_next_id[1, :next_id] + 1

1003159

In [76]:
df_prp_r[!, :id] .= collect(next_id:(next_id + size(df_prp_r, 1) - 1));

In [77]:
df_prp_r[1001:1007, [:id, :person_id, :ID_Domherr, :Amtsart, :ID_Amt]]

Row,id,person_id,ID_Domherr,Amtsart,ID_Amt
,Int64,Int32?,Int64,String?,Int64
1,1004159,659270,45715,missing,82794
2,1004160,659270,45715,missing,82795
3,1004161,659269,45716,missing,82796
4,1004162,659268,45717,missing,82797
5,1004163,659268,45717,missing,82798
6,1004164,659267,45718,missing,82799
7,1004165,659267,45718,missing,82800


In [78]:
columns = [
    :id => :id,
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :Institution => :institution_name,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [79]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prp_r, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: 20000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: 30000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1193
┌ Info: Rows inserted: 39454
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


39454

Titularbistum ist noch nicht eingetragen

## Zeitspanne der Person

In [80]:
table_name = "person"
sql = "SELECT id, num_date_birth, num_date_death
FROM $(table_name) WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [81]:
size(df_date)

(26249, 3)

Zeitspanne der Ämter

In [82]:
df_pr_begin_0 = dropmissing(df_prp_r, :num_date_begin)
df_pr_begin = groupby(select(df_pr_begin_0, :person_id, :num_date_begin), :person_id)

Row,person_id,num_date_begin
,Int32?,Int64
1,636609,1680
2,636609,1678
3,636609,1677
Row,person_id,num_date_begin
,Int32?,Int64
1,662857,1456
2,662857,1473
3,662857,1459
4,662857,1459


In [83]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [84]:
size(df_pr_date_min)

(25332, 2)

In [85]:
df_pr_date_min[3015:3021, :]

Row,person_id,date_min
,Int32?,Int64
1,639646,1440
2,639647,1434
3,639648,1441
4,639649,1375
5,639650,1437
6,639651,1435
7,639652,1439


`date_max`

In [86]:
df_pr_end_value = dropmissing(df_prp_r, :num_date_end)
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id)

Row,person_id,num_date_end
,Int32?,Int64
1,636609,1690
2,636609,1690
3,636609,1690
Row,person_id,num_date_end
,Int32?,Int64
1,662857,1478
2,662857,1478
3,662857,1478
4,662857,1478


In [87]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [88]:
size(df_pr_date_max)

(17810, 2)

In [89]:
df_pr_date_max[3015:3021, :]

Row,person_id,date_max
,Int32?,Int64
1,640541,1382
2,640542,1376
3,640543,1360
4,640545,1327
5,640546,1316
6,640547,1320
7,640548,1314


In [90]:
df_pr_date_min_mng = subset(df_pr_date_min, :person_id => ByRow(ismissing));

In [91]:
size(df_pr_date_min_mng)

(0, 2)

In [92]:
subset!(df_pr_date_min, :person_id => ByRow(!ismissing));

In [93]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [94]:
subset!(df_pr_date_max, :person_id => ByRow(!ismissing));

In [95]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [96]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [97]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [98]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [99]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [100]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [101]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [102]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein

In [103]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [104]:
stmt = DBInterface.prepare(Wds.dbwiag, "UPDATE person SET date_min = ?, date_max = ? WHERE id = ?");

In [105]:
n_loop = 1
for row in eachrow(df_date)
    date_min, date_max, id = row[[:date_min, :date_max, :id]]
    DBInterface.execute(stmt, (date_min, date_max, id))
    n_loop += 1
end
n_loop

25407

In [106]:
DBInterface.close!(stmt)

*2022-01-17* Es gibt etliche Einträge für Personen mit Ämtern nach 2000?! 
```sql
select p.id, id_in_source, item_type_id, givenname, familyname, date_min, date_max, edit_status, is_online
from person as p join item on item.id = p.id and p.date_max > 2000;
```

## Archidiakonat

In [101]:
println.(names(df_prp_r));

Bistum
Amtsart
Amtsbeginn
Amtsende
ID_Amt
Institution
Amt_weltlich
Herrschaftsgebiet
Ort
Amt_Kommentar
Sort
Titularbistum
Profession
ID_Kloster
ID_Fremdschlüssel
Dignität
ID_Domherr
Archidiakonat
Hilfsfeld_import
ID_Amt_norm
id_in_source
person_id
match_role_id
role_name
role_id
diocese_id
institution_id
name
institution_name
num_date_begin
num_date_end
date_sort_key
date_sort_key_replace
id


In [107]:
columns = [
    :id => :person_role_id,
    :person_id => :person_id,
    :Archidiakonat => :value
]

3-element Vector{Pair{Symbol, Symbol}}:
            :id => :person_role_id
     :person_id => :person_id
 :Archidiakonat => :value

In [108]:
df_pr_property = select(df_prp_r, columns);

In [109]:
dropmissing!(df_pr_property, :value);

In [110]:
size(df_pr_property)

(672, 3)

In [111]:
df_pr_property[!, :name] .= "Archidiakonat";

In [112]:
df_pr_property[407:414, :]

Row,person_role_id,person_id,value,name
,Int64,Int32?,String,String
1,1036736,660472,Lemgo,Archidiakonat
2,1036741,660470,Höxter,Archidiakonat
3,1039916,646690,Ostgau,Archidiakonat
4,1039918,646690,Hildesheim,Archidiakonat
5,1040058,662709,Hildesheim,Archidiakonat
6,1040061,662709,Ostgau,Archidiakonat
7,1040168,659313,Merzen,Archidiakonat
8,1040173,652793,Borsum,Archidiakonat


In [113]:
table = "role_property_type"
sql = "SELECT * from $(table)"
df_rpt = Wds.sql_df(sql);

In [114]:
df_rpt

Row,id,name,label,comment,item_type_id
,Int32,String,String,String?,Int32
1,2,Archidiakonat,Archidiakonat,missing,5
2,3,bishpric_seat,Bischofssitz,missing,4
3,4,insignia,Insignien,Besonderheiten,4


In [115]:
rec_archidiakonat = filter(:name => isequal("Archidiakonat"), df_rpt)
archidiakonat_id = rec_archidiakonat[1, :id]

2

In [116]:
insertcols!(
    df_pr_property,
    :property_type_id => archidiakonat_id,
);

In [117]:
table_name = "person_role_property";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person_role_property\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 672, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [118]:
table_name = "person_role_property"
Wds.filltable!(table_name, df_pr_property)

┌ Info: Rows inserted: 672
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


672

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-17* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12